In [10]:
import gym
from env.WarthogEnv import WarthogEnv
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common import make_vec_env
from stable_baselines3.common.utils import set_random_seed
import numpy as np
%matplotlib
from matplotlib import pyplot as plt
from stable_baselines3 import PPO
import torch

Using matplotlib backend: TkAgg


In [ ]:
from typing import Callable

def make_env(env_id: str, rank: int, seed: int = 0) -> Callable:
    """
    Utility function for multiprocessed env.
    
    :param env_id: (str) the environment ID
    :param num_env: (int) the number of environment you wish to have in subprocesses
    :param seed: (int) the inital seed for RNG
    :param rank: (int) index of the subprocess
    :return: (Callable)
    """
    def _init() -> gym.Env:
        env = WarthogEnv('unity_remote.txt')
        env = gym.make(env_id)
        env.seed(seed + rank)
        return env
    set_random_seed(seed)
    return _init

In [ ]:
env_id = "CartPole-v1"
num_cpu = 2  # Number of processes to use
    # Create the vectorized environment
env = SubprocVecEnv([make_env(env_id, i) for i in range(num_cpu)])


In [11]:
env = WarthogEnv('unity_remote.txt')

In [ ]:
#env = DummyVecEnv([lambda: WarthogEnv('unity_remote.txt')])
#env = WarthogEnv('unity_remote.txt')
x = []
y = []
env.reset()
for i in range(0,5000):
    action = [0.5,0.1]
    obs, reward, done, info = env.step(action)
    env.render()
    #x.append(info[0][0])
    #y.append(info[0][1])
    if done:
        #print("resetting")
        env.reset()

In [ ]:
env = WarthogEnv('unity_remote.txt')
#obs = env.reset()
#print(obs)
model = PPO('MlpPolicy', env, verbose=1)

In [ ]:
model.learn(total_timesteps=1e7)

In [ ]:
x = []
y = []
env.reset()
for i in range(0,1000):
    action = [0.5,0.1]
    obs, reward, done, info = env.step(action)
    env.render()
    #x.append(info[0][0])
    #y.append(info[0][1])
    if done:
        print("resetting")
        env.reset()
    #print(info)


In [ ]:
print(env.num_waypoints)

In [ ]:
plt.plot(x,y,'+r')
plt.ylim(-3, 9)
plt.xlim(9, 21)

In [ ]:
model.save("bicycle4")

In [26]:
env = WarthogEnv('unity_remote.txt')

In [ ]:
model1 = PPO('MlpPolicy', env, verbose=1)
#for learning uncomment
model = PPO('MlpPolicy', env, verbose=1)
# model.load('./first_pytorch_multiplication_reward.zip')
model = PPO.load('bicycle4')
model.env = model1.env
model.learn(total_timesteps=1e7)

In [28]:
model1 = PPO('MlpPolicy', env, verbose=1)
model = PPO('MlpPolicy', env, verbose=1)
model = PPO.load('bicycle4.zip')
model.env = model1.env
act1 = []
act2 = []
reward = 0
with torch.no_grad():
    model.policy.eval()
    #envg = model.get_env()
    obs = env.reset()
    #t1 = time.time()
    for i in range(5000):
      #  t2 = time.time()
        #action, _states = model.predict(obs, deterministic=False) 
        action, _states = model.predict(obs)
       # print(action)
        act1.append(np.clip(action[0], 0 ,1)*4)
        act2.append(np.clip(action[1], -1 ,1)*2.5)
        #act2.append(reward) 
        #action[0] = np.clip(action[0], 0, 1)*4
        #action[1] = np.clip(action[1], -1, 1)*2.5
        obs, reward, done, info = env.step(action)
        #print(t2-t1)
        #if t2 -t1 < 0.3:
           # time.sleep(0.3 - (t2-t1))
        #t1 = t2
    #print(action)
        env.render()
        if done:
          obs = env.reset()

Using cpu device
Wrapping the env in a DummyVecEnv.
Using cpu device
Wrapping the env in a DummyVecEnv.


ValueError: unsupported pickle protocol: 5

In [21]:
plt.plot(act1)
plt.show()

In [25]:
plt.plot(act2)
plt.show()